# 🗃️ 数据库调试笔记本

本笔记本用于调试和测试数据库相关功能：
- 数据库连接测试
- 表结构查看
- 数据查询和操作
- 性能分析
- 数据迁移测试

In [ ]:
# 环境设置
import sys
import os
from pathlib import Path

# 添加项目根目录到路径
project_root = Path(os.getcwd()).parent
sys.path.insert(0, str(project_root))

# 设置必要的环境变量（测试用）
os.environ.setdefault("TELEGRAM_BOT_TOKEN", "test_token")
os.environ.setdefault("OPENAI_API_KEY", "test_key")

print(f"项目根目录: {project_root}")
print("✅ 环境设置完成")

## 1. 数据库连接测试

In [ ]:
from src.database import engine, get_db_session, init_db
from sqlalchemy import inspect, text

# 测试连接
try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT 1"))
        print("✅ 数据库连接成功!")
except Exception as e:
    print(f"❌ 数据库连接失败: {e}")

## 2. 查看表结构

In [ ]:
inspector = inspect(engine)
tables = inspector.get_table_names()

print(f"📋 数据库中的表 ({len(tables)} 个):")
print("=" * 60)

for table in tables:
    columns = inspector.get_columns(table)
    indexes = inspector.get_indexes(table)
    
    print(f"\n📌 {table} ({len(columns)} 列, {len(indexes)} 索引)")
    print("-" * 40)
    
    for col in columns:
        nullable = "可空" if col['nullable'] else "非空"
        default = f" = {col['default']}" if col.get('default') else ""
        print(f"   • {col['name']}: {col['type']} ({nullable}){default}")

## 3. 查询数据

In [ ]:
from src.models.database import User, Bot, Channel, ChannelBotMapping, Conversation

db = get_db_session()

try:
    # 统计数据
    print("📊 数据统计:")
    print(f"   👤 用户数: {db.query(User).count()}")
    print(f"   🤖 Bot 数: {db.query(Bot).count()}")
    print(f"   💬 Channel 数: {db.query(Channel).count()}")
    print(f"   🔗 绑定数: {db.query(ChannelBotMapping).count()}")
    print(f"   💭 对话数: {db.query(Conversation).count()}")
finally:
    db.close()

In [ ]:
# 查看所有用户
db = get_db_session()
try:
    users = db.query(User).all()
    print("👤 用户列表:")
    for u in users:
        print(f"   [{u.id}] @{u.username} | {u.first_name} | tier: {u.subscription_tier}")
finally:
    db.close()

In [ ]:
# 查看所有Bot
db = get_db_session()
try:
    bots = db.query(Bot).all()
    print("🤖 Bot 列表:")
    for b in bots:
        token_status = "✅" if b.bot_token and not b.bot_token.startswith("PENDING") else "⏳"
        print(f"   [{b.id}] @{b.bot_username} | {b.bot_name} | {token_status}")
        print(f"       AI: {b.ai_provider}/{b.ai_model}")
finally:
    db.close()

## 4. 性能分析

In [ ]:
import time

def benchmark_query(name: str, query_func, iterations: int = 100):
    """测试查询性能"""
    times = []
    for _ in range(iterations):
        start = time.time()
        query_func()
        times.append(time.time() - start)
    
    avg_time = sum(times) / len(times) * 1000
    max_time = max(times) * 1000
    min_time = min(times) * 1000
    
    print(f"📊 {name}:")
    print(f"   平均: {avg_time:.2f}ms | 最小: {min_time:.2f}ms | 最大: {max_time:.2f}ms")

# 测试各种查询
db = get_db_session()
try:
    benchmark_query("User.count()", lambda: db.query(User).count())
    benchmark_query("Bot.all()", lambda: db.query(Bot).all())
    benchmark_query("Channel.count()", lambda: db.query(Channel).count())
finally:
    db.close()

## 5. 数据操作示例

In [ ]:
# 创建测试用户（示例）
from src.models.database import SubscriptionTier

def create_test_user(telegram_id: int, username: str, first_name: str):
    """创建测试用户"""
    db = get_db_session()
    try:
        # 检查是否已存在
        existing = db.query(User).filter(User.telegram_id == telegram_id).first()
        if existing:
            print(f"⚠️ 用户已存在: {existing.username}")
            return existing
        
        user = User(
            telegram_id=telegram_id,
            username=username,
            first_name=first_name,
            subscription_tier=SubscriptionTier.FREE.value,
            is_active=True
        )
        db.add(user)
        db.commit()
        db.refresh(user)
        print(f"✅ 用户创建成功: {user.username} (ID: {user.id})")
        return user
    except Exception as e:
        db.rollback()
        print(f"❌ 创建失败: {e}")
        return None
    finally:
        db.close()

# 示例：创建测试用户
# create_test_user(123456789, "test_user", "Test")

## 6. 原始SQL查询

In [ ]:
# 执行原始SQL查询
def execute_sql(sql: str):
    """执行SQL并返回结果"""
    with engine.connect() as conn:
        result = conn.execute(text(sql))
        return result.fetchall()

# 示例：查询Bot和绑定数量
sql = """
SELECT b.id, b.bot_username, b.bot_name, COUNT(m.id) as binding_count
FROM bots b
LEFT JOIN channel_bot_mappings m ON b.id = m.bot_id
GROUP BY b.id
ORDER BY binding_count DESC
"""

try:
    results = execute_sql(sql)
    print("Bot 绑定统计:")
    for row in results:
        print(f"   @{row[1]}: {row[3]} 个绑定")
except Exception as e:
    print(f"查询失败: {e}")

## 7. 数据清理工具

In [ ]:
def cleanup_old_conversations(days: int = 30):
    """清理旧对话记录"""
    from datetime import datetime, timedelta
    
    cutoff_date = datetime.utcnow() - timedelta(days=days)
    
    db = get_db_session()
    try:
        count = db.query(Conversation).filter(
            Conversation.timestamp < cutoff_date
        ).count()
        
        print(f"📊 {days} 天前的对话记录: {count} 条")
        
        if count > 0:
            confirm = input(f"是否删除这 {count} 条记录? (yes/no): ")
            if confirm.lower() == "yes":
                db.query(Conversation).filter(
                    Conversation.timestamp < cutoff_date
                ).delete()
                db.commit()
                print(f"✅ 已删除 {count} 条旧记录")
            else:
                print("❌ 已取消")
    finally:
        db.close()

# 预览要清理的数据（不执行删除）
# cleanup_old_conversations(30)